In [ ]:
!pip install python-chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.2 MB/s eta 0:00:00


In [ ]:
import chess.pgn

def evaluate_position(board):
    # Make a copy of the board to ensure the original board state is not modified
    board_copy = board.copy()

    # Piece values
    piece_values = {
        chess.KING: 200,
        chess.QUEEN: 9,
        chess.ROOK: 5,
        chess.BISHOP: 3,
        chess.KNIGHT: 3,  # Assuming both knights have the same value
        chess.PAWN: 1,
    }

    material_score = 0
    # Calculate the count of each piece type for both sides
    for piece_type, value in piece_values.items():
        white_count = len(board_copy.pieces(piece_type, chess.WHITE))
        black_count = len(board_copy.pieces(piece_type, chess.BLACK))
        material_score += value * (white_count - black_count)

    # Doubled pawns, blocked pawns, and isolated pawns evaluation
    white_pawns = list(board_copy.pieces(chess.PAWN, chess.WHITE))
    black_pawns = list(board_copy.pieces(chess.PAWN, chess.BLACK))

    white_doubled_pawns = sum(white_pawns.count(square) > 1 for square in white_pawns)
    black_doubled_pawns = sum(black_pawns.count(square) > 1 for square in black_pawns)

    white_blocked_pawns = sum(board_copy.piece_at(square + 8) is not None for square in white_pawns)
    black_blocked_pawns = sum(board_copy.piece_at(square - 8) is not None for square in black_pawns)

    white_isolated_pawns = sum(
        board_copy.piece_at(square - 1) is None and board_copy.piece_at(square + 1) is None
        for square in white_pawns
    )
    black_isolated_pawns = sum(
        board_copy.piece_at(square - 1) is None and board_copy.piece_at(square + 1) is None
        for square in black_pawns
    )

    pawn_score = 0.5 * (
        white_doubled_pawns - black_doubled_pawns +
        white_blocked_pawns - black_blocked_pawns +
        white_isolated_pawns - black_isolated_pawns
    )

    # Mobility score
    white_mobility = len(list(board_copy.legal_moves))
    board_copy.turn = chess.BLACK  # Switch to opponent's turn to calculate their mobility
    black_mobility = len(list(board_copy.legal_moves))
    board_copy.turn = chess.WHITE  # Switch back to the original turn

    mobility_score = 0.1 * (white_mobility - black_mobility)

    # Calculate the relative score based on who is to move
    who2move = 1 if board_copy.turn == chess.WHITE else -1

    return (material_score + mobility_score + pawn_score) * who2move



#Implement the minimax algorithm with alpha-beta pruning

def negaMax(depth, board, alpha, beta):
    if depth == 0 or board.is_game_over():
        return evaluate_position(board)

    legal_moves = list(board.legal_moves)
    best_score = float('-inf')

    for move in legal_moves:
        board.push(move)
        score = -negaMax(depth - 1, board, -beta, -alpha)
        board.pop()

        if score >= beta:
            return score  # Beta cutoff
        if score > best_score:
            best_score = score
        alpha = max(alpha, score)

    return best_score


# Function to find the best move based on the current board state and evaluation function
def find_best_move(board, depth):
    best_score = float('-inf')
    alpha = float('-inf')
    beta = float('inf')
    best_move = None

    legal_moves = list(board.legal_moves)
    if not legal_moves:
        # Stalemate or draw
        return None

    for move in legal_moves:
        board.push(move)
        score = -negaMax(depth - 1, board, -beta, -alpha)
        board.pop()

        if score > best_score:
            best_score = score
            best_move = move

        alpha = max(alpha, score)

    return best_move

def play_move(board, max_depth):
    if board.turn == chess.WHITE:
        best_move = find_best_move(board, depth=max_depth)
    else:
        best_move = find_best_move(board, depth=max_depth)

    # Check if the game is already over (checkmate or stalemate)
    if board.is_game_over():
        return board

    # Play the move on a copy of the board to avoid modifying the original board
    board_copy = board.copy()
    board_copy.push(best_move)

    return board_copy



In [ ]:
import chess
import chess.pgn

# Define the evaluation functions for white and black bots

# Define the initial board position
initial_board = chess.Board()

# Define the game result (e.g., "1-0" for white win, "0-1" for black win, "1/2-1/2" for a draw)
game_result = None

# Define the maximum number of moves to play
max_moves = 200

# Define the maximum number of moves to play
max_moves = 200

# Initialize a variable to keep track of whose turn it is (0 for white, 1 for black)
current_turn = 0

# Game loop
for move_number in range(max_moves):
    if current_turn == 0:
        # White bot's turn
        white_bot_evaluation = evaluate_position(initial_board)
        print('white_eval:', white_bot_evaluation)
        white_move = find_best_move(initial_board, depth=4)
        print("White's move:", white_move)
        initial_board.push(white_move)
    else:
        # Black bot's turn
        black_bot_evaluation = evaluate_position(initial_board)
        print('black_eval:', black_bot_evaluation)
        black_move = find_best_move(initial_board, depth=3)
        print("Black's move:", black_move)
        initial_board.push(black_move)

    # Toggle the current_turn variable to switch turns
    current_turn = 1 - current_turn

    # Check if the game is over
    if initial_board.is_game_over():
        game_result = initial_board.result()
        break

    # Print a newline for clarity
    print()


# If the game result is still None after max_moves, consider it a draw
if game_result is None:
    game_result = "1/2-1/2"

# Print the game result
print("Game result:", game_result)



In [ ]:
pawn_eval = [
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0],
    [1.0, 1.0, 2.0, 3.0, 3.0, 2.0, 1.0, 1.0],
    [0.5, 0.5, 1.0, 2.5, 2.5, 1.0, 0.5, 0.5],
    [0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 0.0],
    [0.5, -0.5, -1.0, 0.0, 0.0, -1.0, -0.5, 0.5],
    [0.5, 1.0, 1.0, -2.0, -2.0, 1.0, 1.0, 0.5],
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

pawn_eval_black = [row[::-1] for row in pawn_eval[::-1]]

knight_eval = [
    [-5.0, -4.0, -3.0, -3.0, -3.0, -3.0, -4.0, -5.0],
    [-4.0, -2.0, 0.0, 0.0, 0.0, 0.0, -2.0, -4.0],
    [-3.0, 0.0, 1.0, 1.5, 1.5, 1.0, 0.0, -3.0],
    [-3.0, 0.5, 1.5, 2.0, 2.0, 1.5, 0.5, -3.0],
    [-3.0, 0.0, 1.5, 2.0, 2.0, 1.5, 0.0, -3.0],
    [-3.0, 0.5, 1.0, 1.5, 1.5, 1.0, 0.5, -3.0],
    [-4.0, -2.0, 0.0, 0.5, 0.5, 0.0, -2.0, -4.0],
    [-5.0, -4.0, -3.0, -3.0, -3.0, -3.0, -4.0, -5.0]]

knight_eval_black = [row[::-1] for row in knight_eval[::-1]]

bishop_eval = [
    [-2.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -2.0],
    [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0],
    [-1.0, 0.0, 0.5, 1.0, 1.0, 0.5, 0.0, -1.0],
    [-1.0, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, -1.0],
    [-1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, -1.0],
    [-1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0],
    [-1.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, -1.0],
    [-2.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -2.0]]

bishop_eval_black = [row[::-1] for row in bishop_eval[::-1]]

rook_eval = [
    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5],
    [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
    [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
    [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
    [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
    [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
    [0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0]]

rook_eval_black = [row[::-1] for row in rook_eval[::-1]]

queen_eval= [
    [-2.0, -1.0, -1.0, -0.5, -0.5, -1.0, -1.0, -2.0],
    [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0],
    [-1.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, -1.0],
    [-0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, -0.5],
    [0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, -0.5],
    [-1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, -1.0],
    [-1.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, -1.0],
    [-2.0, -1.0, -1.0, -0.5, -0.5, -1.0, -1.0, -2.0]]

queen_eval_black = [row[::-1] for row in queen_eval[::-1]]

king_eval = [
    [-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
    [-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
    [-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
    [-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
    [-2.0, -3.0, -3.0, -4.0, -4.0, -3.0, -3.0, -2.0],
    [-1.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -1.0],
    [2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0],
    [2.0, 3.0, 1.0, 0.0, 0.0, 1.0, 3.0, 2.0]]

king_eval_black = [row[::-1] for row in king_eval[::-1]]



def evaluation(piece_type, piece_color):
    if piece_color == chess.WHITE:
        if piece_type == chess.PAWN:
            return pawn_eval
        elif piece_type == chess.BISHOP:
            return bishop_eval
        elif piece_type == chess.KNIGHT:
            return knight_eval
        elif piece_type == chess.ROOK:
            return rook_eval
        elif piece_type == chess.QUEEN:
            return queen_eval
        elif piece_type == chess.KING:
            return king_eval
    else:
        if piece_type == chess.PAWN:
            return pawn_eval_black
        elif piece_type == chess.BISHOP:
            return bishop_eval_black
        elif piece_type == chess.KNIGHT:
            return knight_eval_black
        elif piece_type == chess.ROOK:
            return rook_eval_black
        elif piece_type == chess.QUEEN:
            return queen_eval_black
        elif piece_type == chess.KING:
            return king_eval_black